# HIV Model
Gail Romer

In [71]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *


In [72]:
"""
Α α, Β β, Γ γ, Δ δ, Ε ε, Ζ ζ, Η η, Θ θ, Ι ι, Κ κ, Λ λ, Μ μ, Ν ν, Ξ ξ, Ο ο, Π π, Ρ ρ, Σ σ/ς, Τ τ, Υ υ, Φ φ, Χ χ, Ψ ψ, Ω ω
"""

'\nΑ α, Β β, Γ γ, Δ δ, Ε ε, Ζ ζ, Η η, Θ θ, Ι ι, Κ κ, Λ λ, Μ μ, Ν ν, Ξ ξ, Ο ο, Π π, Ρ ρ, Σ σ/ς, Τ τ, Υ υ, Φ φ, Χ χ, Ψ ψ, Ω ω\n'

In [73]:
# state = State(R = cd4_lymphocytes, E = actively_effective_cells, L = latently_affected_cells, V = free_virons)
# system = System(Γ = SOMETHING_a, #constraint 1 of 'birthrate' of R
#                 τ = SOMETHING_b, #constraint 2 of 'birthrate' of R
#                 μ = SOMETHING_c, #deathrate of R and L (proportional to R and L themselves)
#                 Β = SOMETHING_e, #transitionrate from R to either E or L 
#                                  #Thats a beta, copy paste
#                 ρ = SOMETHING_d, #transitionrate of ΒR to L (proportional to free virons) (1-rho is transition rate of BR to E) 
#                                  #Thats a rho, copy paste
#                 α = SOMETHING_f, #transitionrate of L to E
#                 δ = SOMETHING_g, #deathrate of E (proportional to E itself)
#                 π = SOMETHING_h, #'birthrate' of V (proportional to E) 
#                 σ = SOMETHING_i, #deathrate of V (proportional to V itself)
#                )


cd4_lymphocytes = 100;
actively_effective_cells = 27;
latently_affected_cells = 34;
free_virons=263



state = State(R = cd4_lymphocytes, L = latently_affected_cells, E = actively_effective_cells, V = free_virons)
system = System(Γ = 1.36,               #Per Day
                                            #constraint 1 of 'birthrate' of R
                τ = 0.2,                #Per Day
                                            #constraint 2 of 'birthrate' of R
                μ = 1.36/(10^-3),       #Per Day                             
                                            #deathrate of R and L (proportional to R and L themselves)
                Β = 0.00027,            #Per Viron
                                            #transitionrate from R to either E or L 
                    #Thats a beta, copy paste
                ρ = 0.1,                #Per Day
                                            #transitionrate of ΒR to L (proportional to free virons) (1-rho is transition rate of BR to E) 
                α = 3.6*(10^-2),        #Per Day
                                            #transitionrate of L to E
                δ = 0.33,               #Per Day
                                            #deathrate of E (proportional to E itself)
                π = 100,                #Per Day
                                            #'birthrate' of V (proportional to E) 
                σ = 2,                   #Per Day 
                                            #deathrate of V (proportional to V itself)
                t_0 = 0,
                
                dt = 0.1
               );

In [74]:
def update_func(state, t, system):
    """INFORMATION
    """
#     r = Γ*τ - μ*state.R - Β*state.R*state.V;
#     l = ρ*Β*state.R*state.V - μ*state.L - α*state.L;
#     e = (1-ρ)*Β*state.R*state.V + α*state.L - δ*state.E;
#     v = π*state.E - σ*state.V;
    
    unpack(state)
    unpack(system)
#     R = Γτ - μR - ΒRV;
    r_production = Γ*τ - μ*R
    r_elimination = Β*R*V
#     L = ρΒRV - μL - αL;
    l_production = ρ*Β*R*V
    l_elimination = μ*L + α*L
#     E = (1-ρ)ΒRV + αL - δE;
    e_production = (1-ρ)*Β*R*V + α*L
    e_elimination = δ*E 
#     V = πE - σV;
    v_production = π*E
    v_elimination = σ*V
    
    
    R += (r_production - r_elimination) * dt
    L += (l_production - l_elimination) * dt 
    E += (e_production - e_elimination) * dt
    V += (v_production - v_elimination) * dt 
   
    
    return State(R=r, L=l, E=e, V=v)

In [75]:
def run_simulation(system, update_func):
    """Runs a simulation of the system.
        
    system: System object
    update_func: function that updates state
    
    returns: TimeFrame
    """
    unpack(system)
    
    frame = TimeFrame(columns=init.index)
    frame.row[t0] = init
    
    for t in linrange(t0, t_end):
        frame.row[t+1] = update_func(frame.row[t], t, system)
    
    return frame

In [76]:
def slope_func(state, t, system):
    """INFORMATION
    """
    #     r = Γ*τ - μ*state.R - Β*state.R*state.V;
    #     l = ρ*Β*state.R*state.V - μ*state.L - α*state.L;
    #     e = (1-ρ)*Β*state.R*state.V + α*state.L - δ*state.E;
    #     v = π*state.E - σ*state.V;
    
    unpack(state)
    unpack(system)
    # #     R = Γτ - μR - ΒRV;
    #     r_production = Γ*τ - μ*R
    #     r_elimination = Β*R*V
    drdt = (Γ * τ - μ * R) - (Β * R * V)
    # #     L = ρΒRV - μL - αL;
    #     l_production = ρ*Β*R*V
    #     l_elimination = μ*L + α*L
    dldt = (ρ * Β * R * V) - (μ * L + α * L)
    # #     E = (1-ρ)ΒRV + αL - δE;
    #     e_production = (1-ρ)*Β*R*V + α*L
    #     e_elimination = δ*E 
    dedt = ((1-ρ) * Β * R * V + α * L) - (δ * E) 
    # #     V = πE - σV;
    #     v_production = π*E
    #     v_elimination = σ*V
    dvdt = (π * E) - (σ * V)
    
    return drdt, dldt, dedt, dvdt

In [77]:
slope_func(state,0,system)

(8.28211111111111, 1474.647877777778, -1471.3191000000002, 2174.0)

In [78]:
system.set(init=init, t_end=200)

In [79]:
results, details = run_ode_solver(system, slope_func)
details

TypeError: cannot convert dictionary update sequence element #0 to a sequence